We'll assume that $Y = \{y_1, y_2, \ldots, y_n\}$ is governed by a mixture of two Gaussians, call them $f_0$ and $f_1$.  .  This implies that there is an unobservable variable, call it $S = \{s_1, s_2, \ldots, s_n\}$ which specifies which Gaussian $Y$ is drawn from.  We will suppose that $S_t$ can assume values 0 or 1, and we will say that $Y_t \sim f_0$ if $S_t= 0$ and $Y_t \sim f_1$ if $S_t = 1$.  As is, this would be considered a random switching model and $S$ would be a sequence of iid Bernoulli random variables.  But this implies a memory-less swithcing process, but we want to impose that information about $S_t$ can inform us about $S_{t+1}$, thus, it makes sense to describe $S$ as a markov process.  For what follows we will consider S to be an order 1 markov process, which can be entirely characterized by the transition matrix 

$$P = \left[\begin{array}{cc}
p_{0,0}	& p_{0,1} \\ 
p_{1,0}	& p_{1,1}
\end{array}\right] $$

The variable $S_t$ is unobservable, but we can estimate the probabilty of $S_t$ via Bayes rule.  As an example of the first observation in $Y$.  

$$P(S_1 = 0 | Y_1 = y_1) = \frac{P(Y_1 = y_1 | S_1 = 0) \cdot P(S_1 = 0)}{P(Y_1 = y_1)}$$

$$ = \frac{ f_1 \cdot \xi }{\xi\cdot f_1 + (1- \xi)\cdot f_0}$$

after computing the inferences for the regimes at time 1, we can use them to make a forecast for the regime distribution at time 2.  

$$P(S_2 = 0 | Y_1 = y_1) =  P(S_2 = 0 | S_1 = 0, Y_1 = y_1)P(S_1 = 0|Y_1 = y_1) + P(S_2=0| S_1 = 1, Y_1 = y_1)P(S_1 = 1|Y_1 = Y_1) $$

$$ = P(S_2 = 0 |S_1 = 0)P(S_1 = 0 | Y_1 = y_1) + P(S_2 = 0 | S_1 = 1)P(S_1 = 1 | Y_1 = y_1)$$

$$ = p_{0,0}\frac{ P(Y_1 = y_1 | S_1 = 0) P(S_1 = 0)}{P(Y_1 = y_1)} + p_{1,0}\frac{ P(Y_1 = y_1 | S_1 = 1) P(S_1 = 1)}{P(Y_1 = y_1)}$$

$$ = P(S_2 = 0 | Y_1 = y_1) = p_{0,0}P(S_1 = 0 | Y_1 = y_1) + p_{1,0}P(S_1 = 0 | Y_1 = y_1)$$

and anolgously

$$P(S_2 = 1 | Y_1 = y_1) = 1 - P(S_2 = 0 | Y_1 = 1)$$

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
%matplotlib inline

df = pd.read_excel("RSExample_MSCIUS.xls", headers=True)

df["return"].plot()



y = df["return"].values
S = np.zeros(y.shape)

In [2]:
mu0 = 0.04
sig0 = 1.0
mu1 = -0.04
sig1 = 4
regimes = 2
xi = 0.5 # pr(s_1 = 0)
p = np.array([[.8,.2], [.2,.8]])

In [3]:
## initialize stuff
xi_t_given_t = np.zeros((S.shape[0], 2))
xi_tp1_given_t = np.zeros((S.shape[0], 2))

xi_tp1_given_t[0,0] = 0.5
xi_tp1_given_t[0,1] = 0.5

In [20]:
oldLogLike = -3412.8839872
for ii in range(1):
    f0 = norm.pdf(y, mu0, sig0)
    f1 = norm.pdf(y, mu1, sig1)
    f = np.array( [f0, f1] ).T

    # p(s_1 = 0 | Y_1)
    # initialize xi_t given t as follows
    temp_calc = xi*f0[0] / (xi*f0[0] + (1-xi)*f1[0])
    xi_t_given_t[0,:] = np.array([temp_calc, 1 - temp_calc])
    xi_t_given_t

    # p(s_2 = 0 | Y_1) 
    # p(s_2 = 1 | Y_1)
    xi_tp1_given_t[0,0] = xi
    xi_tp1_given_t[0,1] = 1-xi
    for i in range(1,S.shape[0]):
        xi_tp1_given_t[i,:] = xi_t_given_t[i-1,:].dot(p.T)
        xi_t_given_t[i,:] = xi_tp1_given_t[i,:]*f[i,:] / xi_tp1_given_t[i,:].dot(f[i,:])

    # inference probabilities
    print("\ninference probs")
    print(pd.DataFrame(xi_t_given_t).head())


    # forecast probabilites
    print("\nforecast probs")
    print(pd.DataFrame(xi_tp1_given_t).head())

    ## smoothed probabilities
    ## use the inference probabiilty as the intial value, iterate through backware to get the smoothing probabilities

    xi_t_given_T = np.zeros(xi_t_given_t.shape)
    xi_t_given_T[-1,:] = xi_t_given_t[-1,:]

    for i in range(xi_t_given_T.shape[0]-2,-1,-1):    
        temp = xi_t_given_t[i,:] * (p.T.dot( xi_t_given_T[i+1,:] / xi_tp1_given_t[i+1,:]))
        xi_t_given_T[i, :] = temp

    print("\nsmoothed probs")
    print(pd.DataFrame(xi_t_given_T).head())

    # log likelihood
    print("\nlog likelihood")
    print(np.log(np.sum(f * xi_tp1_given_t, axis=1)).sum())

    ## ml estimate of mu0 and mu1
    mu0,mu1 = (xi_t_given_T * y.reshape([y.shape[0],1])).sum(axis=0) / xi_t_given_T.sum(axis=0)

    ## ml estimate of sigma0 and sigma1
    new_sigma_iter = (xi_t_given_T * (y.reshape([y.shape[0],1]) - np.array([mu0, mu1]))**2).sum(axis=0) / xi_t_given_T.sum(axis=0)

#     mu0,mu1 = new_mu_iter
    sigma0, sigma1 = np.sqrt(new_sigma_iter)
    
    p_00 = np.zeros(y.shape)
    p_11 = np.zeros(y.shape)
    p_00[0] = p[0,0]
    p_11[0] = p[1,1]
    for i in range(1,p_00.shape[0]):
        p_00[i] = xi_t_given_t[i-1, 0] * xi_t_given_T[i,0]/xi_tp1_given_t[i,0]*p[0,0]
        p_11[i] = xi_t_given_t[i-1, 1] * xi_t_given_T[i,1]/xi_tp1_given_t[i,1]*p[1,1]

    p_00 = p_00[1:].sum() / xi_t_given_T[0:-1, 0].sum()
    p_11 = p_11[1:].sum() / xi_t_given_T[0:-1, 1].sum()
    p[0,0] = p_00
    p[0,1] = 1 - p_00
    p[1,1] = p_11
    p[1,0] = 1 - p_11

#     print("\ntransition matrix")
#     print(p)

#     mu0,mu1 = new_mu_iter
#     sigma0, sigma1 = np.sqrt(new_sigma_iter)

    xi = xi_t_given_T[0,0]
#     print("\npr(s1 = 0)")
#     print(xi)

#     print("\nmu0 -> {}, sigma0 -> {}".format(mu0, sigma0))
#     print("\nmu1 -> {}, sigma1 -> {}".format(mu1, sigma1))


inference probs
          0         1
0  0.713338  0.286662
1  0.336510  0.663490
2  0.543358  0.456642
3  0.516551  0.483449
4  0.770750  0.229250

forecast probs
          0         1
0  0.572980  0.427020
1  0.651526  0.363652
2  0.383879  0.604490
3  0.530795  0.472289
4  0.511756  0.489422

smoothed probs
          0         1
0  0.572896  0.329446
1  0.474832  0.451589
2  0.601878  0.293347
3  0.640994  0.244627
4  0.715798  0.151456

log likelihood
-3410.95360699


In [12]:
     ## log likelihood
print("\nlog likelihood")
print(np.log(np.sum(f * xi_tp1_given_t, axis=1)).sum())

print("\ntransition matrix")
print(p)

print("\npr(s1 = 0)")
print(xi)

print("\nmu0 -> {}, sigma0 -> {}".format(mu0, sigma0))
print("\nmu1 -> {}, sigma1 -> {}".format(mu1, sigma1))


log likelihood
-3411.63557448

transition matrix
[[ 0.84059255  0.15940745]
 [ 0.20105599  0.79894401]]

pr(s1 = 0)
0.638444749284

mu0 -> 0.2274680816702556, sigma0 -> 1.1584782910494325

mu1 -> -0.24363339106119966, sigma1 -> 3.1725894576494125


In [ ]:
q = pd.DataFrame(xi_t_given_T, columns = ["p0", "p1"])["p0"]
q.index = df["date"]

In [ ]:
q.to

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=[12,10])
q.plot(ax=ax)